In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

In [2]:
# Chemins vers les dossiers d'images et de labels
images_dir = './dataset/images/'
labels_dir = './dataset/labels/'

# Lister tous les fichiers
image_paths = [os.path.join(images_dir, fname) for fname in os.listdir(images_dir) if fname.endswith('.jpg')]
label_paths = [os.path.join(labels_dir, fname) for fname in os.listdir(labels_dir) if fname.endswith('.txt')]

# Assurer que les listes sont triées pour correspondre les images aux labels
image_paths.sort()
label_paths.sort()

In [3]:
image_paths

['./dataset/images/5_10_jpg.rf.5639f7a69885c595fc4b54e55dbf5148.jpg',
 './dataset/images/5_11_jpg.rf.99ee6f27d5d5f353e00506003993302a.jpg',
 './dataset/images/5_12_jpg.rf.f46713998d4579eabe3baf01d0bc605c.jpg',
 './dataset/images/5_13_jpg.rf.d4959906ae25dc3b28e2e43553764b1c.jpg',
 './dataset/images/5_14_jpg.rf.af89c1ce48edc27397830962d43d5c6a.jpg',
 './dataset/images/5_15_jpg.rf.02c45be3c76c4201f24dd386e060ed8c.jpg',
 './dataset/images/5_16_jpg.rf.ac71b8803fe5c99c3fc1e6867351c0bb.jpg',
 './dataset/images/5_17_jpg.rf.0611b10e50517e4323c47fc36301167c.jpg',
 './dataset/images/5_22_jpg.rf.368e2a41bf84be83076f603939cbd052.jpg',
 './dataset/images/5_23_jpg.rf.b98dce2a37d470d921751909d8030cee.jpg',
 './dataset/images/5_24_jpg.rf.36c0b527bff3e31a8effc6d45c8be54a.jpg',
 './dataset/images/5_25_jpg.rf.0a5f2cffb167f51cdfbd781fa2c84799.jpg',
 './dataset/images/5_26_jpg.rf.56a14e695c9d4f114bc7695b4eeee5c4.jpg',
 './dataset/images/5_27_jpg.rf.27629ee7ad1efad4706917a4d365f885.jpg',
 './dataset/images/5

In [4]:
label_paths

['./dataset/labels/5_10_jpg.rf.5639f7a69885c595fc4b54e55dbf5148.txt',
 './dataset/labels/5_11_jpg.rf.99ee6f27d5d5f353e00506003993302a.txt',
 './dataset/labels/5_12_jpg.rf.f46713998d4579eabe3baf01d0bc605c.txt',
 './dataset/labels/5_13_jpg.rf.d4959906ae25dc3b28e2e43553764b1c.txt',
 './dataset/labels/5_14_jpg.rf.af89c1ce48edc27397830962d43d5c6a.txt',
 './dataset/labels/5_15_jpg.rf.02c45be3c76c4201f24dd386e060ed8c.txt',
 './dataset/labels/5_16_jpg.rf.ac71b8803fe5c99c3fc1e6867351c0bb.txt',
 './dataset/labels/5_17_jpg.rf.0611b10e50517e4323c47fc36301167c.txt',
 './dataset/labels/5_22_jpg.rf.368e2a41bf84be83076f603939cbd052.txt',
 './dataset/labels/5_23_jpg.rf.b98dce2a37d470d921751909d8030cee.txt',
 './dataset/labels/5_24_jpg.rf.36c0b527bff3e31a8effc6d45c8be54a.txt',
 './dataset/labels/5_25_jpg.rf.0a5f2cffb167f51cdfbd781fa2c84799.txt',
 './dataset/labels/5_26_jpg.rf.56a14e695c9d4f114bc7695b4eeee5c4.txt',
 './dataset/labels/5_27_jpg.rf.27629ee7ad1efad4706917a4d365f885.txt',
 './dataset/labels/5

In [5]:
# Calculer les indices de séparation
total_images = len(image_paths)
train_split = int(0.8 * total_images)
total_images,train_split

(2516, 2012)

In [6]:
# Mélanger les données (les paires image-label doivent être mélangées de la même manière)
indices = np.arange(total_images)
np.random.shuffle(indices)

train_indices = indices[:train_split]
test_indices = indices[train_split:]
train_indices,test_indices

(array([1093,  507, 1196, ...,    8,  490,   99]),
 array([2295,  382, 2069,  915, 1088, 2340, 2142, 2430, 1534, 2002,  858,
        1951, 1891, 2041,  754,  829,  159,  319, 1962,  586,  278,  943,
         560,   70,  488,  776,  589,  950, 1395, 2480,   63, 1191, 1217,
        2266, 2230, 2138,  892,  509,  756, 2428, 2497,  455,  555,  115,
        1356, 1360, 1398, 1975, 2100, 1218, 2028,  240, 1416, 1480,  136,
         924, 1366,  196, 1129,  889, 1475,  464,  185,  123,  372, 1448,
        1834,  210, 1155, 1374, 1138, 1325,  873,  257, 2199, 2243, 2387,
        2413, 2022, 1090,  312, 2439, 2361, 1386, 1565,  797,  104,  233,
         878, 2215, 1656,  598,  815, 1497, 1712,   45, 1089,  259,  821,
        1280,  448, 2403, 2168,  793, 1405, 1186, 2377, 1014,   98, 1778,
        2277, 2156,  365, 2263,  198,  364, 1342, 1777, 2106,  661, 1553,
        1202, 2250,  116, 2204, 1856, 1855,  500,  405,  834,  824,  928,
         847, 1823, 2359, 2476, 1704,  906, 1796, 2135,  569,

In [7]:
# Assigner les données aux ensembles
train_images = [image_paths[i] for i in train_indices]
train_labels = [label_paths[i] for i in train_indices]

test_images = [image_paths[i] for i in test_indices]
test_labels = [label_paths[i] for i in test_indices]
len(train_images),len(test_labels)

(2012, 504)

In [8]:
def convert_bounding_box(data_str):
    # Convertit la chaîne en liste de flottants
    data = list(map(float, data_str.split(',')))
    
    # Ignorer l'ID de classe qui est le premier élément
    _, x_center, y_center, width, height = data
    
    # Calculer x_min et y_min
    x_min = x_center - (width / 2)
    y_min = y_center - (height / 2)
    
    # Les valeurs [x_min, y_min, width, height] sont renvoyées
    return [x_min, y_min, width, height]

In [9]:
def load_label(label_path):
    """Charge et parse le fichier de label, renvoyant uniquement les coordonnées de la boîte."""
    with open(label_path, 'r') as file:
        data = file.readline().strip()
        if data:
            # Supposer que les données de boîte suivent le format 'class_id, x, y, width, height'
            # et on ne prend que les coordonnées x, y, width, height
            parts = data.split()
            # Convertir seulement les parties coordonnées en flottants (on ignore parts[0] qui est class_id)
            return [float(num) for num in parts[1:]]  # Prend les 4 valeurs de coordonnées
        else:
            return [0.0, 0.0, 0.0, 0.0]  # Valeurs par défaut pour les images sans accident


In [10]:
label = load_label(label_paths[30])
label

[0.3125, 0.35703125, 0.2953125, 0.2953125]

In [11]:
tf.config.list_physical_devices('GPU')

[]

In [12]:
def create_datagen(image_paths, label_paths, batch_size):
    """Génère des lots d'images et de labels."""
    while True:
        for start in range(0, len(image_paths), batch_size):
            end = min(start + batch_size, len(image_paths))
            batch_images, batch_labels = [], []
            for i in range(start, end):
                image = load_img(image_paths[i], target_size=(64, 64))
                image = img_to_array(image) / 255.0
                label = load_label(label_paths[i])
                batch_images.append(image)
                batch_labels.append(label)
            yield np.array(batch_images), np.array(batch_labels)

In [13]:
# Utilisation
train_generator = create_datagen(train_images, train_labels, batch_size=64)
test_generator = create_datagen(test_images, test_labels, batch_size=64)
train_generator,test_generator

(<generator object create_datagen at 0x00000230267B7E00>,
 <generator object create_datagen at 0x0000023026914040>)

In [14]:
def create_cnn(input_shape, num_outputs):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),  # Assurez-vous que cette valeur correspond à la sortie du Flatten
        Dropout(0.5),
        Dense(num_outputs)  # Sorties pour les coordonnées de la boîte
    ])

    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    return model

In [15]:
model = create_cnn((64, 64, 3), 4)
model.summary()

C:\Users\brice\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 64, 64, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 32, 32, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 16, 16, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       4,194,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │           2,052 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,290,116 (16.37 MB)

 Trainable params: 4,290,116 (16.37 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
history = model.fit(train_generator, epochs=1, validation_data=test_generator, verbose=1)
history

  14167/Unknown 11098s 783ms/step - accuracy: 0.6131 - loss: 0.0145

KeyboardInterrupt: 

In [ ]:
# Afficher les courbes de perte
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Loss (training)')
plt.plot(history.history['val_loss'], label='Loss (validation)')
plt.title('Loss Progression')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Afficher les courbes de précision si elles sont disponibles
if 'accuracy' in history.history:
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Accuracy (training)')
    plt.plot(history.history['val_accuracy'], label='Accuracy (validation)')
    plt.title('Accuracy Progression')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

plt.show()


In [ ]:
# Évaluation
val_loss, val_acc = model.evaluate(validation_generator)
print(f'Validation accuracy: {val_acc*100:.2f}%')

In [ ]:
# Faire une prédiction sur une nouvelle image
def load_and_prepare_image(filepath):
    img = image.load_img(filepath, target_size=(64, 64))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.
    return img_tensor

new_image = load_and_prepare_image('./dataset/aug_18_test4_10_jpg.rf.b97252d8abc30ed694e10a0035a7a011.jpg')
prediction = model.predict(new_image)
predicted_class = np.argmax(prediction)


In [17]:
del model